# Merge County-level Tile Indices


In [69]:
import os
import pandas as pd
import geopandas as gpd

### Discover Tile Indices


In [10]:
lidar_path = '../lidar'
counties = sorted(os.listdir(lidar_path))
indices = [os.path.join(lidar_path, c, 'tiles.gpkg') for c in counties 
          if os.path.exists(os.path.join(lidar_path, c, 'tiles.gpkg'))]
print(f'INFO: {len(indices)} tile indices found.')

INFO: 80 tile indices found.


### Merge Tile Indices
Read each tile index into GeoPandas dataframe and concatenate them together.

In [67]:
frames = [gpd.read_file(i) for i in indices]
tindex = pd.concat(frames, ignore_index=True)
tindex.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 267188 entries, 0 to 267187
Data columns (total 5 columns):
location    267188 non-null object
srs         267188 non-null object
modified    267188 non-null object
created     267188 non-null object
geometry    267188 non-null object
dtypes: object(5)
memory usage: 10.2+ MB


### Setting Relative Path
When PDAL creates the tindex it provides the LAS file path in the _location_ field. Unfortuately, the path created is an absolute path made worse because the full path may not exist on any given machine. Therefore, we'll convert it to a relative path. 

The _location_ field can now be used to construct a full path by prepending a path to the lidar folder. Within notebooks this would most likely be '../lidar'. More importantly, the relative path can be combined with a URL to allow remote access of the LAS files.

In [68]:
try:
    tindex['location'] = tindex['location'].apply(lambda i: i[i.index('lidar/')+6:])
except ValueError as e:
    print('ERROR: location missing lidar directory')
tindex.head()

,location,srs,modified,created,geometry
0,adams/las_classified/1280_1930.las,EPSG:26972,2009-10-20T00:37:09,2018-12-13T21:46:57,POLYGON ((-91.47852667224002 40.17411279626138...
1,adams/las_classified/1156_2086.las,EPSG:26972,2009-10-19T22:34:02,2018-12-13T21:46:57,POLYGON ((-90.91659129758993 39.83871065542411...
2,adams/las_classified/2066_1216.las,EPSG:3444,2009-10-19T23:56:13,2018-12-13T21:46:57,"POLYGON ((-90.98977805630592 40.0029290734882,..."
3,adams/las_classified/1966_1234.las,EPSG:3444,2009-10-19T22:18:49,2018-12-13T21:46:57,POLYGON ((-91.34753807035025 40.04924836807506...
4,adams/las_classified/1240_1992.las,EPSG:26972,2009-10-19T23:15:51,2018-12-13T21:46:57,POLYGON ((-91.25494238878736 40.06662605018638...


### List All Projections
The tile index maintains LAS boundaries in geometric coordinates but the LAS files come in a wide variety of projected coordinate system. The native projection of each LAS is stored in the _srs_ field. 

In [74]:
print('\n'.join(sorted(tindex['srs'].unique())))

+proj=tmerc +lat_0=36.66666666666666 +lon_0=-88.33333333333333 +k=0.9999749999999999 +x_0=300000.0000000001 +y_0=0 +ellps=GRS80 +units=us-ft +vunits=us-ft +no_defs 
+proj=tmerc +lat_0=36.6666666666667 +lon_0=-88.3333333333333 +k=0.9999749999999999 +x_0=300000.0000000001 +y_0=0 +ellps=GRS80 +units=us-ft +vunits=us-ft +no_defs 
+proj=tmerc +lat_0=36.6666666666667 +lon_0=-90.1666666666667 +k=0.999941176470588 +x_0=699999.9999999993 +y_0=0 +ellps=GRS80 +units=us-ft +vunits=us-ft +no_defs 
EPSG:26915
EPSG:26916
EPSG:26972
EPSG:3443
EPSG:3444
EPSG:3531
EPSG:6455
EPSG:6457


__NOTE:__ many of the LAS files were missing projection information entirely and a small number even had incorrect projections. See the *proj_fix.ipynb* for details. 

### Write Master Index

In [79]:
tindex.to_file('../tindex.gpkg', driver='GPKG')
print(f'file written: {os.path.getsize("../tindex.gpkg")} bytes')